# Module 5: Long-Term Memory with LangChain

**Transitioning from LangGraph to LangChain:**
- Module 5.3: Built long-term memory with LangGraph Store
- Module 5.4: **Implement similar patterns with pure LangChain!**

**What you'll learn:**
- 🔄 LangGraph → LangChain transition
- 💾 External memory stores (Redis, PostgreSQL)
- 🗂️ Memory backends and integrations
- 📊 Vector stores for semantic memory
- 🔍 Retrieval-based memory
- 🎯 Production patterns with LangChain

**Key Differences:**

| Aspect | LangGraph | LangChain |
|--------|-----------|----------|
| **Store** | Built-in Store API | External integrations |
| **Approach** | Native memory management | Plugin-based memory |
| **Use Case** | Graph-based workflows | Chain-based workflows |
| **Flexibility** | Tightly integrated | More flexible backends |

**Time:** 2-3 hours

## Setup: Install Dependencies

In [ ]:
# Install LangChain and memory backends
!pip install --pre -U langchain langchain-openai langchain-community
!pip install redis  # For Redis memory backend
!pip install faiss-cpu  # For vector-based semantic search
!pip install psycopg2-binary  # For PostgreSQL

## Setup: Configure API Keys & Imports

In [ ]:
from google.colab import userdata
import os
import json
from datetime import datetime

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# LangChain imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from typing import Dict, List, Any, Optional

print("✅ Setup complete!")

---
# Part 1: Custom Memory Store Pattern 🗂️

**LangChain Approach:** Build custom memory stores using external systems

## Lab 1.1: Simple Dictionary-Based Memory Store

**Replicating LangGraph Store API with pure Python**

In [ ]:
from typing import Dict, Any, Tuple, List, Optional
from dataclasses import dataclass, field

@dataclass
class MemoryItem:
    """Item stored in memory."""
    namespace: Tuple[str, ...]
    key: str
    value: Dict[str, Any]
    created_at: str = field(default_factory=lambda: datetime.now().isoformat())
    updated_at: str = field(default_factory=lambda: datetime.now().isoformat())

class SimpleLongTermMemory:
    """Simple long-term memory implementation."""
    
    def __init__(self):
        self._store: Dict[Tuple[Tuple[str, ...], str], MemoryItem] = {}
    
    def put(self, namespace: Tuple[str, ...], key: str, value: Dict[str, Any]) -> None:
        """Save or update a memory item."""
        item_key = (namespace, key)
        
        if item_key in self._store:
            # Update existing
            self._store[item_key].value = value
            self._store[item_key].updated_at = datetime.now().isoformat()
        else:
            # Create new
            self._store[item_key] = MemoryItem(
                namespace=namespace,
                key=key,
                value=value
            )
    
    def get(self, namespace: Tuple[str, ...], key: str) -> Optional[MemoryItem]:
        """Retrieve a memory item."""
        return self._store.get((namespace, key))
    
    def search(self, namespace: Tuple[str, ...]) -> List[MemoryItem]:
        """Search for items matching namespace prefix."""
        results = []
        for (item_ns, item_key), item in self._store.items():
            # Check if item_ns starts with search namespace
            if len(item_ns) >= len(namespace):
                if item_ns[:len(namespace)] == namespace:
                    results.append(item)
        return results
    
    def delete(self, namespace: Tuple[str, ...], key: str) -> bool:
        """Delete a memory item."""
        item_key = (namespace, key)
        if item_key in self._store:
            del self._store[item_key]
            return True
        return False

# Test the custom memory store
memory = SimpleLongTermMemory()

print("=" * 70)
print("Lab 1.1: Custom Memory Store")
print("=" * 70 + "\n")

# Save user profile
memory.put(
    namespace=("users", "profiles"),
    key="user_101",
    value={
        "name": "Priya Sharma",
        "department": "Engineering",
        "preferences": {"notification": "email"}
    }
)
print("✅ Saved user profile")

# Retrieve profile
profile = memory.get(("users", "profiles"), "user_101")
print(f"\n📥 Retrieved: {profile.value['name']} from {profile.value['department']}")

# Save multiple users
memory.put(
    namespace=("users", "profiles"),
    key="user_102",
    value={"name": "Rahul Verma", "department": "Marketing"}
)

# Search
all_profiles = memory.search(("users", "profiles"))
print(f"\n🔍 Found {len(all_profiles)} user profiles")
for p in all_profiles:
    print(f"  - {p.value['name']} ({p.value['department']})")

print("\n✅ Custom memory store working!")

## Lab 1.2: Integrating with LangChain Chains

In [ ]:
# Create memory store
memory_store = SimpleLongTermMemory()

# Pre-populate with user data
memory_store.put(
    namespace=("users",),
    key="user_101",
    value={
        "name": "Priya Sharma",
        "preferences": {
            "communication_style": "professional",
            "meeting_time": "morning"
        },
        "past_issues": [
            "Portal access problem - solved by IT reset"
        ]
    }
)

# Custom function to enrich prompt with memory
def get_enriched_prompt(user_id: str, query: str) -> str:
    """Get prompt enriched with user's long-term memory."""
    
    # Retrieve user profile from memory
    user_profile = memory_store.get(("users",), user_id)
    
    if user_profile:
        profile_data = user_profile.value
        context = f"""User Context (from long-term memory):
- Name: {profile_data.get('name', 'Unknown')}
- Communication style: {profile_data.get('preferences', {}).get('communication_style', 'standard')}
- Preferred meeting time: {profile_data.get('preferences', {}).get('meeting_time', 'flexible')}
- Past issues: {', '.join(profile_data.get('past_issues', []))}
"""
    else:
        context = "No user context available."
    
    full_prompt = f"""{context}

User Query: {query}

Instructions: Respond to the user considering their preferences and past interactions.
"""
    return full_prompt

# Create LangChain chain
llm = ChatOpenAI(model="gpt-4o-mini")

# Test with memory-enriched prompts
print("=" * 70)
print("Lab 1.2: LangChain with Custom Memory")
print("=" * 70 + "\n")

user_query = "I'm having trouble accessing the portal again."
enriched_prompt = get_enriched_prompt("user_101", user_query)

print("Query:", user_query)
print("\nEnriched Prompt:")
print(enriched_prompt)

response = llm.invoke([HumanMessage(content=enriched_prompt)])
print("\nAssistant Response:")
print(response.content)

print("\n✅ LangChain chain using long-term memory!")

---
# Part 2: Redis-Based Memory Store 🔴

**Production Pattern:** Use Redis for distributed, persistent memory

## Lab 2.1: Redis Memory Backend

In [ ]:
import json
from typing import Dict, Any, Tuple, List, Optional

# Note: This is a mock implementation for demo purposes
# In production, use actual Redis client

class RedisLongTermMemory:
    """Redis-backed long-term memory.
    
    In production, initialize with:
    import redis
    self.redis_client = redis.Redis(host='localhost', port=6379, db=0)
    """
    
    def __init__(self, redis_client=None):
        # For demo: use dict. In production: use actual Redis
        self._mock_store = {}
        self.redis_client = redis_client
    
    def _make_key(self, namespace: Tuple[str, ...], key: str) -> str:
        """Create Redis key from namespace and key."""
        ns_str = ":".join(namespace)
        return f"memory:{ns_str}:{key}"
    
    def put(self, namespace: Tuple[str, ...], key: str, value: Dict[str, Any]) -> None:
        """Save to Redis."""
        redis_key = self._make_key(namespace, key)
        
        item = {
            "value": value,
            "namespace": namespace,
            "key": key,
            "updated_at": datetime.now().isoformat()
        }
        
        # In production: self.redis_client.set(redis_key, json.dumps(item))
        self._mock_store[redis_key] = item
    
    def get(self, namespace: Tuple[str, ...], key: str) -> Optional[Dict[str, Any]]:
        """Retrieve from Redis."""
        redis_key = self._make_key(namespace, key)
        
        # In production: data = self.redis_client.get(redis_key)
        data = self._mock_store.get(redis_key)
        
        if data:
            # In production: return json.loads(data)
            return data
        return None
    
    def search(self, namespace: Tuple[str, ...]) -> List[Dict[str, Any]]:
        """Search by namespace pattern."""
        ns_str = ":".join(namespace)
        pattern = f"memory:{ns_str}:*"
        
        # In production: keys = self.redis_client.keys(pattern)
        keys = [k for k in self._mock_store.keys() if k.startswith(f"memory:{ns_str}:")]
        
        results = []
        for key in keys:
            # In production: data = json.loads(self.redis_client.get(key))
            data = self._mock_store[key]
            results.append(data)
        
        return results
    
    def delete(self, namespace: Tuple[str, ...], key: str) -> bool:
        """Delete from Redis."""
        redis_key = self._make_key(namespace, key)
        
        # In production: return self.redis_client.delete(redis_key) > 0
        if redis_key in self._mock_store:
            del self._mock_store[redis_key]
            return True
        return False

# Test Redis memory
redis_memory = RedisLongTermMemory()

print("=" * 70)
print("Lab 2.1: Redis Memory Backend")
print("=" * 70 + "\n")

# Save user preferences
redis_memory.put(
    namespace=("hr", "preferences"),
    key="user_101",
    value={
        "name": "Priya Sharma",
        "notification_channel": "email",
        "work_schedule": "9-5"
    }
)
print("✅ Saved to Redis")

# Retrieve
data = redis_memory.get(("hr", "preferences"), "user_101")
print(f"\n📥 Retrieved: {data['value']['name']}")
print(f"   Notification: {data['value']['notification_channel']}")

# Save multiple preferences
redis_memory.put(
    namespace=("hr", "preferences"),
    key="user_102",
    value={"name": "Rahul Verma", "notification_channel": "slack"}
)

# Search all preferences
all_prefs = redis_memory.search(("hr", "preferences"))
print(f"\n🔍 Found {len(all_prefs)} preference records")
for pref in all_prefs:
    print(f"  - {pref['value']['name']}: {pref['value']['notification_channel']}")

print("\n✅ Redis memory backend working!")
print("💡 In production, replace mock_store with actual Redis client")

---
# Part 3: Vector Store for Semantic Memory 🔍

**Use Case:** Semantic search over past interactions

## Lab 3.1: FAISS Vector Store for Memory Retrieval

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document

# Create embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Sample past interactions
past_interactions = [
    {
        "user": "user_101",
        "query": "How do I apply for leave?",
        "resolution": "Go to HR Portal → Leave → Apply. Fill form and submit to manager.",
        "date": "2024-01-15"
    },
    {
        "user": "user_101",
        "query": "Can't access the leave portal",
        "resolution": "Reset your credentials via IT support. Issue resolved in 10 minutes.",
        "date": "2024-02-03"
    },
    {
        "user": "user_102",
        "query": "How to request work from home?",
        "resolution": "Submit WFH request in Portal → Attendance section. Manager approval required.",
        "date": "2024-02-10"
    },
    {
        "user": "user_103",
        "query": "What's my leave balance?",
        "resolution": "Check Portal → Leave → Balance. Shows current year allocation and used days.",
        "date": "2024-02-20"
    }
]

# Create documents for vector store
documents = []
for interaction in past_interactions:
    # Combine query and resolution for better semantic search
    content = f"Query: {interaction['query']}\nResolution: {interaction['resolution']}"
    
    doc = Document(
        page_content=content,
        metadata={
            "user": interaction["user"],
            "date": interaction["date"],
            "query": interaction["query"]
        }
    )
    documents.append(doc)

# Create FAISS vector store
vector_store = FAISS.from_documents(documents, embeddings)

print("=" * 70)
print("Lab 3.1: Vector Store Memory")
print("=" * 70 + "\n")
print(f"✅ Created vector store with {len(documents)} interactions\n")

# Function to retrieve similar past interactions
def find_similar_interactions(query: str, k: int = 2) -> List[Document]:
    """Find k most similar past interactions."""
    results = vector_store.similarity_search(query, k=k)
    return results

# Test similarity search
test_queries = [
    "I need help with leave application",
    "Having issues logging into the portal",
    "Want to work remotely"
]

for query in test_queries:
    print(f"Query: '{query}'")
    similar = find_similar_interactions(query, k=1)
    
    if similar:
        print(f"Most similar past interaction:")
        print(f"  {similar[0].page_content}")
        print(f"  (from {similar[0].metadata['date']})\n")

print("✅ Vector-based semantic memory retrieval working!")

## Lab 3.2: Memory-Augmented Chain

In [ ]:
# Create a chain that uses vector memory
def create_memory_augmented_response(user_query: str, user_id: str) -> str:
    """Generate response using past interactions as context."""
    
    # 1. Find similar past interactions
    similar_interactions = find_similar_interactions(user_query, k=2)
    
    # 2. Build context from past interactions
    context = "Relevant past interactions:\n"
    for i, doc in enumerate(similar_interactions, 1):
        context += f"{i}. {doc.page_content}\n"
    
    # 3. Create prompt with memory context
    prompt = f"""{context}

Current User Query: {user_query}
User ID: {user_id}

Instructions: Using the past interactions above as reference, provide a helpful response to the current query. 
If similar issues were resolved before, mention how they were solved.
"""
    
    # 4. Generate response
    llm = ChatOpenAI(model="gpt-4o-mini")
    response = llm.invoke([HumanMessage(content=prompt)])
    
    return response.content

print("=" * 70)
print("Lab 3.2: Memory-Augmented Chain")
print("=" * 70 + "\n")

# Test with new query
query = "I'm having trouble with the leave system"
response = create_memory_augmented_response(query, "user_101")

print(f"User Query: {query}")
print(f"\nAssistant Response:")
print(response)

print("\n✅ Chain using vector memory for context!")

---
# Part 4: Hybrid Memory System 🔄

**Combining multiple memory backends**

## Lab 4.1: Complete Hybrid Memory System

In [ ]:
class HybridMemorySystem:
    """Combines structured memory (Redis/Dict) with semantic memory (Vector Store)."""
    
    def __init__(self):
        # Structured memory for profiles, preferences
        self.structured_memory = SimpleLongTermMemory()
        
        # Vector memory for semantic search of interactions
        self.vector_documents = []
        self.embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
        self.vector_store = None
    
    def save_profile(self, user_id: str, profile: Dict[str, Any]):
        """Save user profile to structured memory."""
        self.structured_memory.put(
            namespace=("profiles",),
            key=user_id,
            value=profile
        )
    
    def get_profile(self, user_id: str) -> Optional[Dict[str, Any]]:
        """Retrieve user profile from structured memory."""
        item = self.structured_memory.get(("profiles",), user_id)
        return item.value if item else None
    
    def save_interaction(self, user_id: str, query: str, resolution: str):
        """Save interaction to both structured and vector memory."""
        # 1. Save to structured memory
        interaction_id = f"int_{len(self.vector_documents)}"
        self.structured_memory.put(
            namespace=("interactions", user_id),
            key=interaction_id,
            value={
                "query": query,
                "resolution": resolution,
                "timestamp": datetime.now().isoformat()
            }
        )
        
        # 2. Add to vector store for semantic search
        content = f"Query: {query}\nResolution: {resolution}"
        doc = Document(
            page_content=content,
            metadata={
                "user_id": user_id,
                "interaction_id": interaction_id,
                "timestamp": datetime.now().isoformat()
            }
        )
        self.vector_documents.append(doc)
        
        # Rebuild vector store
        self.vector_store = FAISS.from_documents(self.vector_documents, self.embeddings)
    
    def find_similar_past_issues(self, query: str, k: int = 2) -> List[Document]:
        """Find similar past interactions using vector search."""
        if self.vector_store is None:
            return []
        return self.vector_store.similarity_search(query, k=k)
    
    def generate_personalized_response(self, user_id: str, query: str) -> str:
        """Generate response using both memory types."""
        # 1. Get user profile (structured memory)
        profile = self.get_profile(user_id)
        
        # 2. Find similar past interactions (vector memory)
        similar_interactions = self.find_similar_past_issues(query, k=2)
        
        # 3. Build comprehensive context
        context = ""
        
        if profile:
            context += f"User Profile:\n"
            context += f"- Name: {profile.get('name', 'Unknown')}\n"
            context += f"- Department: {profile.get('department', 'Unknown')}\n"
            prefs = profile.get('preferences', {})
            if prefs:
                context += f"- Preferences: {prefs}\n"
        
        if similar_interactions:
            context += "\nSimilar Past Interactions:\n"
            for i, doc in enumerate(similar_interactions, 1):
                context += f"{i}. {doc.page_content}\n"
        
        # 4. Generate response
        prompt = f"""{context}

Current Query: {query}

Generate a personalized response considering the user's profile and past interactions.
"""
        
        llm = ChatOpenAI(model="gpt-4o-mini")
        response = llm.invoke([HumanMessage(content=prompt)])
        
        return response.content

# Test hybrid system
hybrid_memory = HybridMemorySystem()

print("=" * 70)
print("Lab 4.1: Hybrid Memory System")
print("=" * 70 + "\n")

# Setup: Save user profile
hybrid_memory.save_profile(
    user_id="user_101",
    profile={
        "name": "Priya Sharma",
        "department": "Engineering",
        "preferences": {
            "communication": "professional",
            "notification": "email"
        }
    }
)
print("✅ Saved user profile")

# Save past interactions
hybrid_memory.save_interaction(
    user_id="user_101",
    query="Can't access leave portal",
    resolution="IT reset credentials. Issue resolved."
)
hybrid_memory.save_interaction(
    user_id="user_101",
    query="How to apply for leave?",
    resolution="Portal → Leave → Apply. Submit to manager."
)
print("✅ Saved interaction history\n")

# Generate personalized response
query = "I'm having issues with the portal again"
response = hybrid_memory.generate_personalized_response("user_101", query)

print(f"Query: {query}")
print(f"\nPersonalized Response:")
print(response)

print("\n✅ Hybrid memory system working!")
print("💡 Combines structured data (profiles) + semantic search (interactions)")

---
# Summary: LangGraph vs LangChain Memory

## Feature Comparison

| Feature | LangGraph | LangChain |
|---------|-----------|----------|
| **Built-in Store** | ✅ Yes | ❌ Build custom |
| **Namespace Support** | ✅ Native | 🔨 Implement yourself |
| **External Backends** | Limited | ✅ Many options |
| **Vector Search** | Manual integration | ✅ Built-in support |
| **Flexibility** | Structured | Very flexible |
| **Learning Curve** | Lower | Higher |

## When to Use Each

### Use LangGraph When:
- ✅ Building graph-based workflows
- ✅ Want integrated memory management
- ✅ Need checkpointing + memory together
- ✅ Prefer structured, opinionated approach

### Use LangChain When:
- ✅ Need custom memory backends (Redis, PostgreSQL, MongoDB)
- ✅ Want vector search / semantic memory
- ✅ Building chain-based workflows
- ✅ Need maximum flexibility
- ✅ Integrating with existing systems

## Best Practices

### Memory Organization
```python
# Good namespace structure
namespace = (organization_id, department, user_id)

# Examples:
("acme", "engineering", "user_101")  # User data
("acme", "policies")                  # Company-wide
("global", "examples")                # Shared across all
```

### Hybrid Approach
- **Structured Memory**: Profiles, preferences, settings
- **Vector Memory**: Past interactions, examples, documents
- **Combine both** for best results

### Production Checklist
✅ Use persistent backend (Redis, PostgreSQL, not in-memory)  
✅ Implement memory cleanup/archival  
✅ Monitor memory size and performance  
✅ Add error handling for missing data  
✅ Implement access control per user/org  
✅ Use vector stores for large knowledge bases  
✅ Cache frequently accessed memories  

## Key Takeaways

1. **LangGraph Store**
   - Integrated, easy to use
   - Best for graph-based agents
   - Limited to LangGraph ecosystem

2. **LangChain Memory**
   - Flexible, many backend options
   - Great for custom implementations
   - Requires more setup

3. **Hybrid Systems**
   - Combine structured + semantic memory
   - Use right tool for each use case
   - Production systems often need both

## Next Steps
- Explore specific backends (Redis, PostgreSQL, Pinecone)
- Implement memory cleanup strategies
- Add memory analytics and monitoring
- Build multi-tenant memory systems

---

**Remember:** Choose the approach that fits your architecture!